In [5]:
# Practice problem: Scraping amazon.in to get shoe price data 
import pprint,requests,bs4
import pandas as pd

In [6]:
#constants
url = "https://www.amazon.in/s?k=sport+shoes&qid=1625391361"
param_ref= "&ref=sr_pg_"
param_page= "&page="

In [7]:
def scrape_website(url):
    req = requests.get(url)
    # create a bs4 object
    # To avoid warnings, provide "html5lib" explicitly
    #html.parser
    return bs4.BeautifulSoup(req.content, "lxml")


In [25]:
df = pd.DataFrame(columns=['Id','Brand','Description','Price','Discount','Rating','Total_Reviews','Free_Delivery'])
df.set_index('Id')
product_id=1
#Actual Run
for page in range (1,8): 
    page_string = str(page)
    url = url+param_ref+page_string+param_page+page_string
    response = scrape_website(url)
    
    search = response.find_all("div",  attrs={"id": 'search'})[0]
    product_spans = search.find_all("span",attrs={"class": 'celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results'})
    
    for product in product_spans:
        try:
            brand = product.find("span",attrs={"class":'a-size-base-plus a-color-base'}).string
            description = product.find("span",attrs={"class":'a-size-base-plus a-color-base a-text-normal'}).string
            price = float(product.find("span",attrs={"class":'a-price a-text-price'}).find("span",attrs={"aria-hidden":'true'}).string.replace(',','').replace('₹',''))
            discount  = float(product.find("span",attrs={"class":'a-price'}).find("span",attrs={"aria-hidden":'true'}).find("span",attrs={"class":'a-price-whole'}).string.replace(',',''))
            rating  = float(product.find("span",attrs={"class":'a-icon-alt'}).string.split("out")[0].strip())
            num_of_reviews  = int(product.find("span",attrs={"class":'a-size-base'}).string.replace(',',''))
            delivery_content  = product.find("div",attrs={"class":'a-row a-size-base a-color-secondary s-align-children-center'}).string
            is_free_deliver = delivery_content.lower().count("free delivery") >0 
        except AttributeError:

               is_free_deliver = False
        #row = pd.DataFrame.from_dict({'Brand':brand,'Description':description,'Price':price,'Discount':discount,'Rating':rating,'Number_Of_Reviews':num_of_reviews,'Delivery':delivery_content})
        a_row = [product_id,brand,description,price,discount,rating,num_of_reviews,is_free_deliver]
        #df= pd.concat([df,a_row],ignore_index=True)
        df.loc[len(df)] = a_row
        product_id +=1


In [26]:
print(df.head(10))


   Id                Brand                                        Description  \
0   1                CLYMB          Outdoor Sports Running Shoes for Mens Boy   
1   2                Sparx                            Men SM-482 Sports Shoes   
2   3                Sparx                           mens Sm-602 Running Shoe   
3   4                Sparx                                 Men's Running Shoe   
4   5               Bourge       Men's Loire-61 Navy and D.Grey Running Shoes   
5   6               ALICON  Sports Shoes for Men | Lace up Lightweight Sho...   
6   7          A ANTIC MOP       Black Fashion Sport Men's Running Shoes(901)   
7   8              Sklodge  Men's Air Wonder Series Mesh Sports Casual Sho...   
8   9  WORLD WEAR FOOTWEAR  Men's (9212-9310) Multicolor Casual Sports Run...   
9  10               Ethics                                   mens Shoes Shoes   

    Price  Discount  Rating Total_Reviews Free_Delivery  
0   999.0     299.0     3.0          1680         

In [13]:
df_rating_avg = df.groupby(['Brand']).agg('mean')

In [14]:
print(df_rating_avg.sort_values(['Rating'],ascending=False))

                                Rating
Brand                                 
road runner                   5.000000
ARKS                          5.000000
Tior                          4.500000
Pro                           4.400000
Shoefly                       4.350000
Puma                          4.213333
Coaster                       4.200000
Cogs                          4.200000
ASICS                         4.200000
Bacca Bucci                   4.175000
Liberty                       4.112500
VGOOD                         4.100000
AFROJACK                      4.100000
ACTION                        4.100000
Adidas                        4.085714
Bourge                        4.053333
Sparx                         4.021429
Centrino                      4.000000
ZORIK                         4.000000
Generic                       4.000000
Harmeet                       4.000000
FURO by Redchief              4.000000
Mikaasa                       4.000000
Fusefit                  

In [22]:
df_products = df.groupby(['Brand']).agg('sum')
print(df_products.head(50))

                              Rating
Brand                               
2ROW                             2.6
A ANTIC MOP                      3.3
ACTION                           4.1
AFROJACK                         4.1
ALICON                           2.8
ARKS                             5.0
ASIAN                           53.4
ASICS                            4.2
Action Shoes                    11.9
Adidas                          28.6
Amico                            3.3
Auras                            3.0
BERKINS                          6.2
BLACKTOWN                        7.6
BUCADIA                         10.8
Bacca Bucci                     16.7
Big Fox                          7.3
Bond Street by (Red Tape)        3.6
Bourge                          60.8
CLYMB                           27.1
Campus                         107.3
Centrino                         4.0
Chevit                           3.5
Coaster                          4.2
Cogs                            12.6
C

In [21]:
df_brand_prod = df.groupby(['Brand','Description'])

In [28]:
print(df.pivot_table(index='Brand',values='Total_Reviews',aggfunc='sum').sort_values('Total_Reviews',ascending=False))

                              Total_Reviews
Brand                                      
ASIAN                                 70731
Sparx                                 67519
Bourge                                62033
WORLD WEAR FOOTWEAR                   41236
Campus                                26868
Lotto                                 13866
Adidas                                 9675
Lancer                                 7803
Power                                  7624
Ethics                                 7520
T-Rock                                 7121
Reebok                                 5285
AFROJACK                               5268
Puma                                   3565
Fusefit                                3081
TRASE                                  3067
Inklenzo                               3067
CLYMB                                  2687
Nivia                                  2583
aadi                                   1830
Big Fox                         